In [1]:
import pandas as pd
import numpy as np
GBQ_PROJECT_ID = '620265099307'
pd.options.display.float_format = '{:,.2f}'.format

df1 = pd.read_csv('chem_by_subpara_by_ccg_monthly',dtype={'subpara': object}) # read from local csv as memory issues with pd.io
df1['month'] = df1['month'].str[:10]

q2 = '''SELECT DISTINCT chemical, chemical_code from ebmdatalab.hscic.bnf'''
chem = pd.io.gbq.read_gbq(q2, GBQ_PROJECT_ID, dialect='standard',verbose=False)

#q3 = '''SELECT DISTINCT subpara, subpara_code from ebmdatalab.hscic.bnf'''
#subp = pd.io.gbq.read_gbq(q3, GBQ_PROJECT_ID, dialect='standard',verbose=False)

## Clean the data: sort out null denominators

In [2]:
df1_unstack = df1[['chemical','month','ratio','pct']].set_index(['chemical','month','pct'])
df1_unstack = df1_unstack.unstack()
#df1_unstack.sort_index() #.loc[df1_unstack.index.duplicated(keep=False)]
df1_unstack = df1_unstack.fillna(0)
df1_unstack
data2 = df1_unstack.stack()
data2.head()
#####REMEMBER TO DROP WIERD CCGS!!!!!!!!!!!

ratio
chemical  month      pct       
0101010C0 2010-08-01 00C   0.05
                     00D   0.01
                     00F   0.00
                     00J   0.02
                     00K   0.05

### Filter out low numbers (chemical and subpara)

In [3]:
# total prescribing for each chemical
#sum numerators to find total volume for each chemical
num = pd.DataFrame(df1["num"].groupby(df1["chemical"]).sum()).reset_index()
num["num centile"] = pd.qcut(num["num"], 10, labels=np.arange(1,11,1))
num.head()

'''# total prescribing for each paragraph
d3 = df1_unstack[["pct","subpara","denom_subpara"]].drop_duplicates()
d3 = d3.groupby("subpara").sum().sort_values(by="denom_subpara")
d3["denom centile"] = pd.qcut(d3["denom_subpara"], 10, labels=np.arange(1,11,1))
d3 = d3.reset_index()'''


# merge with data table
data3 = data2.reset_index().merge(num, how="inner", on="chemical",suffixes=("","_total"))
#data3 = data3.merge(d3, how="inner", on="subpara",suffixes=("","_total"))
data3 = data3.loc[(data3["num centile"]>4)]
data3 = data3[['chemical','month','ratio','pct']].set_index(['chemical','month','pct'])
data3.head()

ValueError: Buffer dtype mismatch, expected 'Python object' but got 'long long'

Exception ValueError: "Buffer dtype mismatch, expected 'Python object' but got 'long long'" in 'pandas._libs.lib.is_bool_array' ignored


ratio
chemical  month      pct       
0101010C0 2010-08-01 00C   0.05
                     00D   0.01
                     00F   0.00
                     00J   0.02
                     00K   0.05

In [4]:
kurtosis_stack = data3.reset_index().groupby(['chemical','month']).apply(pd.DataFrame.kurt)
kurtosis_stack = pd.DataFrame(kurtosis_stack)
kurtosis_stack.index = kurtosis_stack.index.droplevel(2)
kurtosis_stack.columns = ['value']
kurtosis = kurtosis_stack[~kurtosis_stack.index.duplicated(keep='first')].unstack()

In [49]:
skew_stack = data3.reset_index().groupby(['chemical','month']).apply(pd.DataFrame.skew)
skew_stack = pd.DataFrame(skew_stack)
skew_stack.index = skew_stack.index.droplevel(2)
skew_stack.columns = ['value']
skew = skew_stack[~skew_stack.index.duplicated(keep='first')].unstack()

In [5]:
stdev_stack = data3.reset_index().groupby(['chemical','month']).std()
stdev_stack.columns = ['value']
stdev = stdev_stack.unstack()
stdev.head()

value                                                         \
month     2010-08-01 2010-09-01 2010-10-01 2010-11-01 2010-12-01 2011-01-01   
chemical                                                                      
0101010C0       0.05       0.06       0.05       0.05       0.07       0.06   
0101010G0       0.20       0.20       0.20       0.21       0.21       0.20   
0101010I0       0.02       0.07       0.07       0.02       0.07       0.02   
0101010J0       0.11       0.07       0.07       0.06       0.09       0.09   
0101010L0       0.13       0.14       0.14       0.14       0.13       0.13   

                                                         ...                 \
month     2011-02-01 2011-03-01 2011-04-01 2011-05-01    ...     2016-11-01   
chemical                                                 ...                  
0101010C0       0.06       0.08       0.06       0.06    ...           0.06   
0101010G0       0.21       0.20       0.20       0.20    ...           0.23   
0101010I0       0.02       0.02       0.02       0.04    ...           0.06   
0101010J0       0.09       0.07       0.11       0.08    ...           0.04   
0101010L0       0.14       0.13       0.14       0.13    ...           0.06   

                                                                             \
month     2016-12-01 2017-01-01 2017-02-01 2017-03-01 2017-04-01 2017-05-01   
chemical                                                                      
0101010C0       0.07       0.07       0.06       0.07       0.06       0.07   
0101010G0       0.24       0.24       0.24       0.24       0.25       0.24   
0101010I0       0.05       0.05       0.05       0.05       0.05       0.05   
0101010J0       0.04       0.04       0.04       0.04       0.04       0.03   
0101010L0       0.07       0.07       0.07       0.07       0.06       0.07   

                                            
month     2017-06-01 2017-07-01 2017-08-01  
chemical                                    
0101010C0       0.07       0.07       0.07  
0101010G0       0.24       0.25       0.25  
0101010I0       0.05       0.05       0.05  
0101010J0       0.04       0.04       0.03  
0101010L0       0.07       0.07       0.07  

[5 rows x 85 columns]

In [18]:
mean_stack = data3.reset_index().groupby(['chemical','month']).mean()
mean_stack.columns = ['value']
mean = mean_stack.unstack()
mean.head()

value                                                         \
month     2010-08-01 2010-09-01 2010-10-01 2010-11-01 2010-12-01 2011-01-01   
chemical                                                                      
0101010C0 0.03       0.03       0.03       0.03       0.03       0.03         
0101010G0 0.31       0.31       0.31       0.31       0.33       0.31         
0101010I0 0.01       0.01       0.01       0.01       0.02       0.01         
0101010J0 0.09       0.08       0.08       0.08       0.09       0.09         
0101010L0 0.16       0.16       0.17       0.16       0.16       0.16         

                                                         ...                 \
month     2011-02-01 2011-03-01 2011-04-01 2011-05-01    ...     2016-11-01   
chemical                                                 ...                  
0101010C0 0.03       0.03       0.03       0.03          ...     0.03         
0101010G0 0.31       0.30       0.30       0.30          ...     0.38         
0101010I0 0.01       0.01       0.01       0.01          ...     0.03         
0101010J0 0.08       0.08       0.09       0.08          ...     0.03         
0101010L0 0.16       0.16       0.17       0.16          ...     0.06         

                                                                             \
month     2016-12-01 2017-01-01 2017-02-01 2017-03-01 2017-04-01 2017-05-01   
chemical                                                                      
0101010C0 0.04       0.04       0.03       0.04       0.04       0.04         
0101010G0 0.39       0.38       0.39       0.40       0.38       0.38         
0101010I0 0.03       0.03       0.03       0.03       0.02       0.03         
0101010J0 0.03       0.03       0.03       0.03       0.02       0.02         
0101010L0 0.06       0.06       0.07       0.07       0.06       0.07         

                                            
month     2017-06-01 2017-07-01 2017-08-01  
chemical                                    
0101010C0 0.04       0.04       0.04        
0101010G0 0.38       0.39       0.39        
0101010I0 0.03       0.03       0.03        
0101010J0 0.02       0.02       0.02        
0101010L0 0.06       0.07       0.06        

[5 rows x 85 columns]

# CUSUM

In [6]:
def cusum(df, months_smoothing = 12, sensitivity = 5):
    
    # remove date labels from columns
    dates = df.columns #save date labels for later
    df.columns = range(df.shape[1])

    # create necessary tables with same index
    smax = pd.DataFrame(index=df.index)
    smin = pd.DataFrame(index=df.index)
    reference_value = pd.DataFrame(index=df.index)
    threshold = pd.DataFrame(index=df.index)
    alert = pd.DataFrame(index=df.index)
    not_alert = pd.DataFrame(index=df.index)

    # set initial values for s
    smax[0] = 0
    smin[0] = 0

    # set initial reference value for each 
    reference_value[0] = df.loc[:,0:months_smoothing-1].mean(axis=1)
    #set initial threshold
    threshold[0] = df.loc[:,0:months_smoothing-1].std(axis=1, ddof=0) * sensitivity

    # loop over months of data
    for i in range(1,df.shape[1]):

        # logic to determine alerts from previous month
        alert = (smax[i-1] > threshold[i-1]) | (smin[i-1] < -threshold[i-1])
        not_alert = (smax[i-1] <= threshold[i-1]) & (smin[i-1] >= -threshold[i-1])
        ## I'm sure there's a way of doing without this ^ one,
            # just coudn't work out the syntax to slice pandas rows
            # based on the opposite of the array of boolean values

        # reference value and threshold:
        ##append previous value if no alert
        reference_value.loc[not_alert, i] = reference_value.loc[not_alert, i-1]
        threshold[i] = threshold[i-1]

        ## reset reference after alert
        reference_value.loc[alert, i] = df.loc[alert,i-months_smoothing:i-1].mean(axis=1)

        # CUSUM calculation (adding on previous s values done elsewhere)
        ##threshold[[i]] / sensitivity is a way of deriving stdev without generating it again
        smax[i] = df[i] - (reference_value[i] + (0.5 * threshold[i] / sensitivity))
        smin[i] = df[i] - (reference_value[i] - (0.5 * threshold[i] / sensitivity))

        # retrigger logic
        retrigger_pos = (smax[i] > 0) & (smax[i-1] > threshold[i-1])
        retrigger_neg = (smin[i] < 0) & (smin[i-1] < -threshold[i-1])
        cont_cumulative = not_alert|retrigger_pos|retrigger_neg
        thres_res = alert!=retrigger_pos|retrigger_neg

        # reset threshold
        threshold.loc[thres_res, i] = df.loc[thres_res,i-months_smoothing:i-1].std(axis=1, ddof=0) * sensitivity

        # add s from previous month if no trigger, or retrigger
        smax.loc[cont_cumulative, i] = smax.loc[cont_cumulative, i] + smax.loc[cont_cumulative, i-1]
        smin.loc[cont_cumulative, i] = smin.loc[cont_cumulative, i] + smin.loc[cont_cumulative, i-1]

        # set baselines of zero
        smax.loc[smax[i] < 0, i] = 0
        smin.loc[smin[i] > 0, i] = 0

    # determine &export alerts
    alert_pos = (smax > threshold)
    #alert_pos.to_csv('alert_pos_pandas.csv')

    alert_neg = (smin < -threshold)
    #alert_neg.to_csv('alert_neg_pandas.csv')
    return {'smax':smax, 'smin':smin,'reference_value':reference_value,
           'threshold':threshold, 'alert':alert, 'alert_pos':alert_pos,
           'alert_neg':alert_neg, 'dates':dates}
# other potentially useful outputs (for drawing graphs etc)
#smax
#smin
#value
#reference_value
#threshold

In [7]:
import base64
import requests
from time import sleep
from itertools import chain
from cStringIO import StringIO
from datetime import timedelta, date
from IPython.display import display, HTML

%pylab inline

# Turn off the max column width so the HTML 
# image tags don't get truncated 
pd.set_option('display.max_colwidth', -1)

# Turning off the max column will display all the data in
# our arrays so limit the number of element to display
pd.set_option('display.max_seq_items', 2)

def sparkline(data, figsize=(4, 0.25), **kwags):
    """
    Returns a HTML image tag containing a base64 encoded sparkline style plot
    """
    data = list(data)
    
    fig, ax = plt.subplots(1, 1, figsize=figsize, **kwags)
    ax.plot(data)
    for k,v in ax.spines.items():
        v.set_visible(False)
    ax.set_xticks([])
    ax.set_yticks([])    

    plt.plot(len(data) - 1, data[len(data) - 1], 'r.')

    ax.fill_between(range(len(data)), data, len(data)*[min(data)], alpha=0.1)
    
    img = StringIO()
    plt.savefig(img)
    img.seek(0)
    plt.close()
    return '<img src="data:image/png;base64,{}"/>'.format(base64.b64encode(img.read()))

Populating the interactive namespace from numpy and matplotlib


In [41]:
def get_data(df):
    cusum_out = cusum(df)
    change_from = cusum_out['reference_value'].copy()
    change = pd.DataFrame(index=df.index)
    change[0] = 0.0
    change_proportion = pd.DataFrame(index=df.index)
    change_proportion[0] = 0.0
    duration = pd.DataFrame(index=df.index)
    duration[0] = 0
    smax = cusum_out['smax']
    smin = cusum_out['smin']
    dates = cusum_out['dates']
    
    for i in range(1,df.shape[1]):
        change_from[i].loc[(smax[i]!=0) | (smin[i]!=0)] = change_from[i-1]

        change[i] = (change_from[i]) - (change_from[i-1])
        change_proportion[i] = (change_from[i] - change_from[i-1]) /change_from[i-1]

        duration[i] = 1
        duration[i].loc[(smax[i-1]>0.0001) | (smin[i-1]<-0.0001)] = duration[i-1] +1
    change.columns = dates
    
    change_proportion.columns = dates
    duration.columns = dates

    output = change.stack().reset_index()
    #print output
    output = output.merge(change_proportion.stack().reset_index(),
                          how='left',
                          on=['chemical','level_1'])
    output = output.merge(duration.stack().reset_index(),
                          how='left',
                          on=['chemical','level_1'])
    output.columns = ['code','month','absolute_change','proportion_change','duration']
    output = output.merge(chem,
                          how='left',
                          left_on='code',
                          right_on='chemical_code')

    output = output.set_index(['code'])

    pd.options.display.float_format = '{:,.2f}'.format
    return output

In [35]:
def make_table(df, sort='proportion_change',sort_ascending=False,table=kurtosis_stack):
    output = get_data(df)
    output2 = output.sort_values(by=sort,
                               ascending=sort_ascending).head(20)
    spl = output2.merge(table.reset_index(),
                        how='left',
                        left_index=True,
                        right_on='chemical')
    spl = spl.groupby(['chemical']).aggregate({'value': sparkline})
    output2 = output2.merge(spl,
                          how='left',
                          left_index=True,
                          right_index=True)

    return HTML(output2.to_html(escape=False))

# Results

## Change in kurtosis

In [42]:
make_table(kurtosis,sort='absolute_change',table=kurtosis_stack)

,month,absolute_change,proportion_change,duration,chemical,chemical_code,value
code,,,,,,,
1310011AA,44,157.78,2.62,23,Retapamulin,1310011AA,
1001022G0,83,119.09,4.02,15,Hydrocortisone Acetate,1001022G0,
0301012S0,30,105.03,15.63,22,Orciprenaline Sulfate,0301012S0,
0202010V0,26,100.45,3.64,13,Metolazone,0202010V0,
1202010L0,63,98.65,2.18,15,Flunisolide,1202010L0,
0102000L0,60,92.93,9.74,48,Glycopyrronium Bromide,0102000L0,
0407020H0,19,88.60,1.49,13,Dipipanone Hydrochloride,0407020H0,
0606020C0,63,83.62,6.19,15,Etidronate Disodium,0606020C0,
1305020A0,23,82.87,15.91,19,Acitretin,1305020A0,


In [13]:
make_table(kurtosis,sort='proportion_change',table=kurtosis_stack)

,month,absolute_change,proportion_change,duration,chemical,chemical_code,value
code,,,,,,,
0501050B0,26,-0.23,127.67,14,Clarithromycin,0501050B0,
1310011M0,57,-0.31,66.64,11,Mupirocin,1310011M0,
1311010I0,61,24.13,40.64,58,Isopropyl Alcohol,1311010I0,
0703010G0,65,7.43,36.71,4,Combined Ethinylestradiol 35mcg,0703010G0,
0905012I0,54,0.51,19.25,20,Cinacalcet Hydrochloride,0905012I0,
0202080K0,75,26.10,16.87,26,Furosemide/Potassium,0202080K0,
1305020A0,23,82.87,15.91,19,Acitretin,1305020A0,
0301012S0,30,105.03,15.63,22,Orciprenaline Sulfate,0301012S0,
0403030P0,60,17.24,15.63,7,Paroxetine Hydrochloride,0403030P0,


## Skew

In [52]:
make_table(skew,sort='absolute_change',table=skew_stack)

,month,absolute_change,proportion_change,duration,chemical,chemical_code,value
code,,,,,,,
1310011AA,44,7.53,1.09,23,Retapamulin,1310011AA,
0501120Q0,70,6.99,2.88,26,Norfloxacin,0501120Q0,
1001022G0,83,6.94,1.62,15,Hydrocortisone Acetate,1001022G0,
0401010W0,30,6.33,1.30,9,Zaleplon,0401010W0,
0606020C0,63,5.90,1.80,15,Etidronate Disodium,0606020C0,
1305020A0,22,5.69,2.92,13,Acitretin,1305020A0,
0702010I0,25,5.38,2.15,15,Oestrogens Conjugated,0702010I0,
0102000L0,60,5.32,2.24,51,Glycopyrronium Bromide,0102000L0,
0404000L0,29,5.25,1.57,14,Dexamfetamine Sulfate,0404000L0,


In [54]:
make_table(skew,sort='proportion_change',table=skew_stack)

,month,absolute_change,proportion_change,duration,chemical,chemical_code,value
0103030S0,80,-0.64,11.47,14,Sucralfate,0103030S0,
0203020I0,36,-0.23,3.28,33,Flecainide Acetate,0203020I0,
0302000C0,47,-0.83,3.25,43,Beclometasone Dipropionate,0302000C0,
0304010D0,58,-0.60,2.99,36,Loratadine,0304010D0,
0403030Q0,78,-0.96,3.51,35,Sertraline Hydrochloride,0403030Q0,
0406000J0,38,-0.86,20.50,18,Domperidone,0406000J0,
0407020A0,48,-0.24,4.60,12,Fentanyl,0407020A0,
0408010Q0,56,-0.50,4.15,56,Phenytoin Sodium,0408010Q0,
0501015P0,55,-0.19,4.15,10,Pivmecillinam Hydrochloride,0501015P0,
0501050H0,66,0.13,12.18,17,Erythromycin Ethylsuccinate,0501050H0,


## Standard deviation

In [44]:
make_table(stdev,sort='proportion_change',table=stdev_stack)

,month,absolute_change,proportion_change,duration,chemical,chemical_code,value
0202010L0,42,0.00,2.78,1,Hydrochlorothiazide,0202010L0,
0202040V0,32,0.00,1.80,16,Triamterene With Thiazides,0202040V0,
0204000F0,27,0.00,2.79,4,Atenolol With Diuretic,0204000F0,
0208020X0,48,0.02,39.81,36,Dabigatran Etexilate,0208020X0,
0301011X0,22,0.00,1.81,14,Indacaterol Maleate,0301011X0,
0401010B0,48,0.01,3.38,13,Chloral Hydrate,0401010B0,
0401010F0,21,0.01,1.79,13,Clomethiazole,0401010F0,
0408020W0,38,0.20,3.98,29,Midazolam Hydrochloride,0408020W0,
0501070X0,76,0.20,7.86,62,Rifaximin,0501070X0,
0604020M0,19,0.01,2.85,2,Testosterone Enantate,0604020M0,


In [32]:
make_table(stdev,sort='absolute_change',table=stdev_stack)

,month,absolute_change,proportion_change,duration,chemical,chemical_code,value
code,,,,,,,
0501070X0,76,0.20,7.86,62,Rifaximin,0501070X0,
0408020W0,38,0.20,3.98,29,Midazolam Hydrochloride,0408020W0,
1103030W0,70,0.20,1.77,20,Ganciclovir,1103030W0,
1311010A0,33,0.16,0.56,33,Alcohol,1311010A0,
0501012G0,46,0.12,0.54,46,Flucloxacillin Sodium,0501012G0,
0602010V0,48,0.12,0.51,26,Levothyroxine Sodium,0602010V0,
0101021B0,47,0.11,0.50,33,Alginic Acid Compound Preparations,0101021B0,
0301011R0,47,0.11,0.56,34,Salbutamol,0301011R0,
090504700,41,0.11,0.37,19,Other Selenium Preps,090504700,


## Change in mean

In [27]:
make_table(mean,sort='proportion_change',table=mean_stack)

,month,absolute_change,proportion_change,duration,chemical,chemical_code,value
code,,,,,,,
0208020X0,76,0.02,152.14,12,Dabigatran Etexilate,0208020X0,
0408020W0,49,0.48,18.77,40,Midazolam Hydrochloride,0408020W0,
0904010F0,72,0.00,14.13,15,Gluten Free/Low Protein Bread,0904010F0,
1201010E0,76,0.01,12.41,42,Betamethasone Sodium Phosphate,1201010E0,
1304000C0,28,0.00,9.84,1,Beclometasone Dipropionate,1304000C0,
1103030W0,70,0.40,9.52,27,Ganciclovir,1103030W0,
1310040M0,67,0.12,8.72,26,Malathion,1310040M0,
1404000W0,54,0.00,7.65,26,Measles. Mumps & Rubella Vaccine,1404000W0,
0803042R0,71,0.01,7.32,5,Degarelix,0803042R0,


In [28]:
make_table(mean,sort='absolute_change',table=mean_stack)

,month,absolute_change,proportion_change,duration,chemical,chemical_code,value
0205052N0,29,0.10,0.42,22,Losartan Potassium,0205052N0,
0208010L0,52,0.12,0.72,52,Dalteparin Sodium,0208010L0,
0301012F0,18,0.11,0.26,12,Ephedrine Hydrochloride,0301012F0,
0303030B0,18,0.12,0.42,18,Roflumilast,0303030B0,
0408020W0,49,0.48,18.77,40,Midazolam Hydrochloride,0408020W0,
0703050B0,74,0.14,3.96,68,Ulipristal Acetate (Emergency Cont),0703050B0,
0803020H0,72,0.14,0.33,19,Medroxyprogesterone Acetate,0803020H0,
0901011F0,28,0.10,0.29,21,Ferrous Fumarate,0901011F0,
090504700,67,0.37,3.43,45,Other Selenium Preps,090504700,
090602500,75,0.21,3.55,45,Other Riboflavine (B2) Preps,090602500,


## Mean change

In [47]:
make_table(mean,sort='proportion_change',sort_ascending=False,table=mean_stack)

,month,absolute_change,proportion_change,duration,chemical,chemical_code,value
code,,,,,,,
0208020X0,76,0.02,152.14,12,Dabigatran Etexilate,0208020X0,
0408020W0,49,0.48,18.77,40,Midazolam Hydrochloride,0408020W0,
0904010F0,72,0.00,14.13,15,Gluten Free/Low Protein Bread,0904010F0,
1201010E0,76,0.01,12.41,42,Betamethasone Sodium Phosphate,1201010E0,
1304000C0,28,0.00,9.84,1,Beclometasone Dipropionate,1304000C0,
1103030W0,70,0.40,9.52,27,Ganciclovir,1103030W0,
1310040M0,67,0.12,8.72,26,Malathion,1310040M0,
1404000W0,54,0.00,7.65,26,Measles. Mumps & Rubella Vaccine,1404000W0,
0803042R0,71,0.01,7.32,5,Degarelix,0803042R0,


In [45]:
make_table(mean,sort='proportion_change',sort_ascending=True,table=mean_stack)

,month,absolute_change,proportion_change,duration,chemical,chemical_code,value
0212000AE,42,-0.00,-0.96,13,Nicotinic Acid/Laropiprant,0212000AE,
0212000U0,25,-0.00,-0.95,15,Nicotinic Acid,0212000U0,
0212000V0,57,-0.00,-0.95,20,Omega-3 Marine Triglycerides,0212000V0,
0212000V0,71,-0.00,-0.94,1,Omega-3 Marine Triglycerides,0212000V0,
0304010AE,71,-0.00,-0.95,19,Rupatadine Fumarate,0304010AE,
0407041F0,65,-0.01,-0.97,57,Ergotamine Tartrate,0407041F0,
0410020A0,27,-0.19,-0.94,20,Bupropion Hydrochloride,0410020A0,
0501120Q0,52,-0.02,-0.95,27,Norfloxacin,0501120Q0,
0505010H0,45,-0.07,-0.97,22,Piperazine,0505010H0,
0603020Z0,46,-0.00,-0.98,46,Triamcinolone Acetonide,0603020Z0,


In [48]:
make_table(mean,sort='absolute_change',sort_ascending=False,table=mean_stack)

,month,absolute_change,proportion_change,duration,chemical,chemical_code,value
0205052N0,29,0.10,0.42,22,Losartan Potassium,0205052N0,
0208010L0,52,0.12,0.72,52,Dalteparin Sodium,0208010L0,
0301012F0,18,0.11,0.26,12,Ephedrine Hydrochloride,0301012F0,
0303030B0,18,0.12,0.42,18,Roflumilast,0303030B0,
0408020W0,49,0.48,18.77,40,Midazolam Hydrochloride,0408020W0,
0703050B0,74,0.14,3.96,68,Ulipristal Acetate (Emergency Cont),0703050B0,
0803020H0,72,0.14,0.33,19,Medroxyprogesterone Acetate,0803020H0,
0901011F0,28,0.10,0.29,21,Ferrous Fumarate,0901011F0,
090504700,67,0.37,3.43,45,Other Selenium Preps,090504700,
090602500,75,0.21,3.55,45,Other Riboflavine (B2) Preps,090602500,


In [46]:
make_table(mean,sort='absolute_change',sort_ascending=True,table=mean_stack)

,month,absolute_change,proportion_change,duration,chemical,chemical_code,value
code,,,,,,,
0106030P0,35,-0.62,-0.77,26,Liquid Paraffin,0106030P0,
0410030A0,38,-0.60,-0.70,38,Buprenorphine Hydrochloride,0410030A0,
0502020A0,54,-0.57,-0.94,40,Ketoconazole,0502020A0,
0606010T0,44,-0.51,-0.87,23,Calcitonin (salmon),0606010T0,
0408020V0,49,-0.47,-0.58,41,Midazolam Maleate,0408020V0,
0601023B0,78,-0.30,-0.75,67,Pioglitazone Hydrochloride,0601023B0,
1310040M0,41,-0.28,-0.96,33,Malathion,1310040M0,
1103030C0,83,-0.26,-0.33,35,Aciclovir,1103030C0,
1001010C0,64,-0.25,-0.79,55,Diclofenac Sodium,1001010C0,
